In [14]:
import statsmodels.api as sm
import pandas as pd

In [16]:
df = pd.read_csv('synthetic_formula.csv')
df.head()

,Id,AI declaration,Date of birth,Source of wealth salary,Source of wealth investment gain,Source of wealth inheritance,Source of wealth rental income,Source of wealth business revenue,Source of wealth other,Source of funds salary,Source of funds investment gain,Source of funds inheritance,Source of funds rental income,Source of funds business revenue,Source of funds other,Days to capital call payment,Source of capital call payment,GDP growth rate,Interest rate,default
0,2106,False,25.12.1965,False,False,False,False,False,Crypto holdings,False,True,False,False,True,Pension,27,Grant,10%,-7%,False
1,7824,True,23.11.1993,True,False,True,False,False,Royalty earnings,True,True,False,True,False,Pension,148,Gift,4%,-6%,False
2,3428,True,25.01.1992,False,False,False,True,False,IP ownership,True,False,True,True,True,Pension,73,Grant,6%,3%,False
3,2378,True,31.10.1994,False,True,True,True,False,Royalty earnings,False,True,True,False,False,Gift,16,Grant,5%,-2%,True
4,4701,True,11.10.1961,False,True,True,False,False,IP ownership,True,False,False,False,False,Pension,180,Grant,-9%,-9%,False


GDP growth rate study - min / max

Synthetic data generation
- Check with what is realistic and possible
- E.g. investment behaviour across countries 
- Start with a region 

For data generation
- Consider the distribution across time / space

In [9]:
def calculate_age(dob):
    # dob is in the format "dd.mm.yyyy"
    birth_year = int(dob.split('.')[2])
    current_year = 2024  # Assume the current year is 2024 for the calculation
    age = current_year - birth_year
    return age


df['Age'] = df['Date of birth'].apply(calculate_age)
df = df.drop(columns=['Date of birth'])

In [10]:
# Convert categorical columns into dummy variables
df_encoded = pd.get_dummies(df, columns=["Source of wealth other", "Source of funds other"], drop_first=True)

# Convert boolean columns to numerical (0 for False, 1 for True) for statistical analysis
df_encoded = df_encoded.replace({True: 1, False: 0})

# Define the target variable 'default' and the features (now encoded)
X_encoded = df_encoded.drop(columns=['Id', 'default'])
y_encoded = df_encoded['default']

# Add a constant to the model (intercept)
# X_encoded = sm.add_constant(X_encoded)


/var/folders/pj/3bkj7rdx41j_rvx83ys41kth0000gn/T/ipykernel_46421/1303654562.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_encoded = df_encoded.replace({True: 1, False: 0})


In [11]:
X_encoded.head()

,AI declaration,Source of wealth salary,Source of wealth investment gain,Source of wealth inheritance,Source of wealth rental income,Source of wealth business revenue,Source of funds salary,Source of funds investment gain,Source of funds inheritance,Source of funds rental income,Source of funds business revenue,Age,Source of wealth other_IP ownership,Source of wealth other_Royalty earnings,Source of funds other_Grant,Source of funds other_Pension
0,0,1,0,1,0,1,1,0,0,1,0,30,0,1,0,0
1,0,1,1,0,0,1,0,0,1,0,0,53,0,0,0,1
2,1,1,0,0,0,1,1,0,1,0,0,28,0,1,0,1
3,1,0,0,0,1,0,0,0,1,1,0,33,0,0,1,0
4,0,0,1,0,0,0,1,1,0,0,1,28,1,0,0,1


In [12]:
# Fit a logistic regression model to see the significance of each feature
logit_model_encoded = sm.Logit(y_encoded, X_encoded)
result_encoded = logit_model_encoded.fit()

Optimization terminated successfully.
         Current function value: 0.329470
         Iterations 8


In [13]:
# Get the summary of the logistic regression model
summary_encoded = result_encoded.summary()
print(summary_encoded)

                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                  100
Model:                          Logit   Df Residuals:                       84
Method:                           MLE   Df Model:                           15
Date:                Wed, 02 Oct 2024   Pseudo R-squ.:                  0.2506
Time:                        11:28:28   Log-Likelihood:                -32.947
converged:                       True   LL-Null:                       -43.967
Covariance Type:            nonrobust   LLR p-value:                    0.1068
                                              coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
AI declaration                             -0.5361      0.708     -0.757      0.449      -1.923       0.851
Source of wealth salary                    -